In [2]:
from src.download_and_extract_model import download_and_extract_model

In [3]:
download_and_extract_model("20170512-110547", "./data")

Extracting model to ./data


In [5]:
!ls data

20170512-110547
20170512-110547.zip
images
learning_rate_retrain_tripletloss.txt
learning_rate_schedule_classifier_casia.txt
learning_rate_schedule_classifier_msceleb.txt
pairs.txt


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import argparse
from src import facenet
import os
import sys
import math
import pickle
from sklearn.svm import SVC

In [2]:
sess = tf.InteractiveSession()


In [3]:
np.random.seed(seed=42)

dataset = facenet.get_dataset("vgg_face_dataset/files/")

In [4]:
dataset[1].image_paths

['dataset/vgg_face_dataset/files/A.J._Buckley/00000001.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000002.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000003.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000004.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000005.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000006.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000007.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000008.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000010.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000011.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000013.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000014.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000015.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000016.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000017.png',
 'dataset/vgg_face_dataset/files/A.J._Buckley/00000019.png',
 'dataset/vgg_face_datas

In [5]:
# Check that there are at least one training image per class
for cls in dataset:
    assert len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset'       

AssertionError: There must be at least one image for each class in the dataset

In [6]:
paths, labels = facenet.get_image_paths_and_labels(dataset)

print('Number of classes: %d' % len(dataset))
print('Number of images: %d' % len(paths))

Number of classes: 2635
Number of images: 1024


In [7]:
# Load the model
print('Loading feature extraction model')
facenet.load_model("data/20170512-110547")

Loading feature extraction model
Model directory: data/20170512-110547
Metagraph file: model-20170512-110547.meta
Checkpoint file: model-20170512-110547.ckpt-250000
INFO:tensorflow:Restoring parameters from data/20170512-110547/model-20170512-110547.ckpt-250000


In [8]:
# Get input and output tensors
images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
embedding_size = embeddings.get_shape()[1]

In [9]:
# Run forward pass to calculate embeddings
BATCH_SIZE=32
IMAGE_SIZE=160

print('Calculating features for images')
nrof_images = len(paths)
nrof_batches_per_epoch = int(math.ceil(1.0*nrof_images / BATCH_SIZE))
emb_array = np.zeros((nrof_images, embedding_size))
for i in range(nrof_batches_per_epoch):
    start_index = i*BATCH_SIZE
    end_index = min((i+1)*BATCH_SIZE, nrof_images)
    paths_batch = paths[start_index:end_index]
    images = facenet.load_data(paths_batch, False, False, IMAGE_SIZE)
    feed_dict = { images_placeholder:images, phase_train_placeholder:False }
    emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)

Calculating features for images


ValueError: Cannot feed value of shape (32, 256, 256, 3) for Tensor u'input:0', which has shape '(?, 160, 160, 3)'

In [ ]:
# Train classifier
print('Training classifier')
clf_model = SVC(kernel='linear', probability=True)
clf_model.fit(emb_array, labels)

# Create a list of class names
class_names = [ cls.name.replace('_', ' ') for cls in dataset]

In [ ]:
test_image = facenet.load_data(["/test/path.jpg"])

feed_dict = { images_placeholder:test_image, phase_train_placeholder:False }
emb_array = sess.run(embeddings, feed_dict=feed_dict)

predictions = clf_model.predict_proba(emb_array)
best_class_indices = np.argmax(predictions, axis=1)
best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]

for i in range(len(best_class_indices)):
    print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))